In [0]:
from google.colab import drive

In [3]:
drive.mount("/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive/


In [4]:
# !mkdir /gdrive/"My Drive"/EVA_phase2/
%cd /gdrive/"My Drive"/EVA_phase2/


/gdrive/My Drive/EVA_phase2


In [0]:
# !wget --header="Host: s3.amazonaws.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" "http://s3.amazonaws.com/text-datasets/aclImdb.zip" -O "aclImdb.zip" -c

--2020-01-20 05:54:39--  http://s3.amazonaws.com/text-datasets/aclImdb.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60711700 (58M) [application/zip]
Saving to: ‘aclImdb.zip’

aclImdb.zip         100%[===================>]  57.90M  24.6MB/s    in 2.4s    

2020-01-20 05:54:41 (24.6 MB/s) - ‘aclImdb.zip’ saved [60711700/60711700]



In [5]:
!ls datasets/

aclImdb.zip  imdb_dataset.pickle  imdb.npz


In [6]:
import numpy as np
from numpy import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [0]:
with open("datasets/imdb_dataset.pickle","rb") as file:
  data=pickle.load(file)

## data object is a tuple of train and test split 50-50 (25000 sentences)

In [8]:
data[0][1][:10]

['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']

In [0]:
import pandas as pd

In [0]:
texts=data[0][0]+data[1][0]
labels=data[0][1]+data[1][1]

In [0]:
df=pd.DataFrame({"texts":texts,"topics":labels})

In [12]:
df.head()

,texts,topics
0,For anyone who liked the series this movie wil...,pos
1,I sell the dead revolves around convicted grav...,pos
2,This film is a very descent remake of the famo...,pos
3,"Joseph Conrad's timeless novel, Heart of Darkn...",pos
4,The Closer She Gets... is an artful documentar...,pos


In [0]:
maxlen = 100 # cuts off reviews after 100 words
training_samples = 40000 # train only on 200 samples
validation_samples = 10000
max_words = 124252

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)


In [0]:
df["labels"]=df["topics"].map({"pos":1,"neg":0})

In [20]:
df=df.sample(frac=1).reset_index(drop=True)
df.head()

,texts,topics,labels
0,"If ""B"" movies, tired and corny scripts, and go...",neg,0
1,Prue and Piper bring Dr. Griffiths to their ho...,pos,1
2,This documentary was very thorough and exposin...,pos,1
3,THE DEADLY MANTIS certainly won't scare any on...,neg,0
4,"OK, so one night me and some friends decided t...",neg,0


## word_index is a dictionary of word_to_idx

In [21]:
sequences = tokenizer.texts_to_sequences(df["texts"])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data_vectors= pad_sequences(sequences, maxlen=maxlen)

idx_to_word={word_index[i]:i for i in word_index}
idxs_to_text=lambda li_idx:" ".join( [idx_to_word[i] if i in idx_to_word else "#" for i in li_idx] )


Found 124252 unique tokens.


In [22]:
data_vectors[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,    43,   492,    97,  1431,     2,  2089,  3076,     2,
        5240, 24017,  1873,    53,    14,    47,   430,     4,  4602,
        4572,     4,  8083,    23,   125,   430,     4,   728,   474,
         239,   354,    11,   902,   834,     9,     1,  4485, 62696,
         151,   334,  4493,   115,   197,     3,   171,  1621,     4,
        5688], dtype=int32)

In [23]:
print(idxs_to_text(data_vectors[0]),df["topics"][0])

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # if b movies tired and corny scripts and golf carts dressed up as some sort of futuristic mode of transport are your sort of entertainment you'll probably enjoy this otherwise forget it the topless newsreader though completely irrelevant did give a few seconds of amusement neg


In [24]:
print(df["texts"][0])

If "B" movies, tired and corny scripts, and golf carts dressed up as some sort of futuristic mode of transport are your sort of entertainment, you'll probably enjoy this. Otherwise, forget it. The topless newsreader, though completely irrelevant, did give a few seconds of amusement.


In [0]:
len(idx_to_word)

from sklearn.model_selection import train_test_split

In [0]:
def split_data(data_vectors,label_vectors,test_size=0.1):
  x_train,x_test,y_train,y_test=train_test_split(data_vectors,label_vectors,test_size=test_size)
  
  return x_train,y_train,x_test,y_test




In [0]:
x_train,y_train,x_test,y_test=split_data(data_vectors,np.array(df["labels"]))

In [28]:
x_train.shape,y_train.shape

((45000, 100), (45000,))

In [29]:
y_train[:10]

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 0])

## Download and load pretrained word embeddigs (Glove)

In [0]:
# !wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

--2020-01-19 19:29:45--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  2.06MB/s    in 6m 27s  

2020-01-19 19:36:12 (2.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
# !unzip glove.6B.zip
import os

## Loading glove 100d embeddings

In [0]:
embeddings_index = {}
f = open(os.path.join( 'glove/glove.6B.100d.txt'))
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

In [32]:
len(embeddings_index)

400000

In [33]:
embeddings_index["growing"]

array([-3.0191e-01,  8.8384e-01,  3.3208e-01,  1.0923e-01,  1.9068e-01,
       -3.2629e-01, -7.4598e-01, -5.7549e-01,  4.7926e-01,  2.0772e-01,
       -1.2972e+00, -3.8137e-01, -3.9921e-03, -5.3190e-01,  1.6570e-01,
       -3.3231e-01, -3.2595e-01,  8.2377e-02,  6.6829e-02, -1.0943e-01,
       -3.9902e-01,  5.1666e-01,  2.4612e-01,  2.0786e-01,  1.7056e-03,
        2.2861e-01,  5.2369e-01, -7.5865e-01, -3.3429e-02,  4.7330e-01,
       -6.6450e-02,  1.3777e-01,  8.0831e-01, -4.9327e-01, -7.5526e-02,
        4.9018e-01,  5.1670e-01, -1.5269e-01,  1.4589e-01,  2.6081e-01,
       -8.4302e-01, -5.3116e-01,  9.6419e-02, -7.5570e-02,  7.9627e-01,
       -3.8372e-01,  9.0381e-01,  3.7004e-01,  3.1373e-01, -8.7537e-01,
       -2.1100e-02, -3.1616e-01, -2.8370e-01,  1.1182e+00,  3.4998e-01,
       -2.1596e+00, -9.8544e-02, -5.4007e-01,  1.4882e+00, -7.1375e-03,
        3.3755e-01,  6.4208e-01, -6.0304e-02, -3.1889e-01,  4.6451e-01,
        2.3915e-01,  5.5707e-01, -7.2938e-01,  1.3341e+00,  2.45

In [0]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words+1, embedding_dim))

for i,word in idx_to_word.items():
  
  if i < max_words:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

# words not found in the embedding index will all be zeros


In [35]:
embedding_matrix.shape

(124253, 100)

In [0]:
from keras.layers import BatchNormalization,GlobalAveragePooling1D,AveragePooling1D

In [0]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

def model_():

  model = Sequential()
  model.add(Embedding(max_words+1, embedding_dim, input_length=maxlen))

  model.add(GlobalAveragePooling1D())
  model.add(Dense(128, activation='relu'))
  # model.add(AveragePooling1D())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()

  # load Glove model
  model.layers[0].set_weights([embedding_matrix])
  model.layers[0].trainable = True
  return model

In [40]:

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train, 
epochs = 10,
batch_size=128,
validation_data=(x_test, y_test))
model.save_weights('models/pre_trained_glove_model.h5')




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 6s 142us/step - loss: 0.4701 - acc: 0.7700 - val_loss: 0.3709 - val_acc: 0.8348
Epoch 2/10
45000/45000 [==============================] - 4s 98us/step - loss: 0.2903 - acc: 0.8778 - val_loss: 0.3284 - val_acc: 0.8546
Epoch 3/10
45000/45000 [==============================] - 4s 96us/step - loss: 0.2203 - acc: 0.9123 - val_loss: 0.3284 - val_acc: 0.8600
Epoch 4/10
45000/45000 [==============================] - 4s 95us/step - loss: 0.1643 - acc: 0.9388 - val_loss: 0.3675 - val_acc: 0.8548
Epoch 5/10
45000/45000 [==============================] - 4s 97us/step - loss: 0.1225 - acc: 0.9573 - val_loss: 0.3946 - val_acc: 0.8578
Epoch 6/10
45000/45000 [==============================] - 4s 96us/step - loss: 0.0870 - acc: 0.9716 - val_loss: 0.4692 - val_acc: 0.8446
Epoch 7/10
45000/45000 [======

## Now training on 8000 and validating on 42000 samples

In [0]:
x_train,y_train,x_test,y_test=split_data(data_vectors,np.array(df["labels"]),test_size=0.84)

In [44]:
model=model_()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          12425300  
_________________________________________________________________
global_average_pooling1d_2 ( (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               12928     
_________________________________________________________________
dense_5 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 12,442,389
Trainable params: 12,442,389
Non-trainable params: 0
_________________________________________________________________


In [45]:

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train, 
epochs = 10,
batch_size=128,
validation_data=(x_test, y_test))
model.save_weights('models/pre_trained_glove_model_8000_samples.h5')

Train on 8000 samples, validate on 42000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 209us/step - loss: 0.6638 - acc: 0.6135 - val_loss: 0.6162 - val_acc: 0.6833
Epoch 2/10
8000/8000 [==============================] - 1s 154us/step - loss: 0.5251 - acc: 0.7530 - val_loss: 0.4650 - val_acc: 0.7840
Epoch 3/10
8000/8000 [==============================] - 1s 159us/step - loss: 0.3727 - acc: 0.8327 - val_loss: 0.4172 - val_acc: 0.8097
Epoch 4/10
8000/8000 [==============================] - 1s 162us/step - loss: 0.2740 - acc: 0.8850 - val_loss: 0.3899 - val_acc: 0.8276
Epoch 5/10
8000/8000 [==============================] - 1s 156us/step - loss: 0.2025 - acc: 0.9227 - val_loss: 0.4094 - val_acc: 0.8286
Epoch 6/10
8000/8000 [==============================] - 1s 157us/step - loss: 0.1418 - acc: 0.9491 - val_loss: 0.4149 - val_acc: 0.8347
Epoch 7/10
8000/8000 [==============================] - 1s 153us/step - loss: 0.0954 - acc: 0.9716 - val_loss: 0.4430 - val_acc: 0.8351